In [1]:
# import torch as th

import matplotlib.pyplot as plt
import time

import sys
import os
from __future__ import division
import os
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
%matplotlib inline
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches
import cv2
import pandas as pd
from svgpathtools import parse_path
import svg_distance_helpers as rsh
# import torch
# from torch.autograd import Variable
# import SimpleITK as sitk
# from torch.autograd import Variable


#### File Hierarchy

In [ ]:
# directory & file hierarchy
analysis_dir = os.getcwd()
data_dir = os.path.join(analysis_dir,"tracing_overlap")
ref_dir = os.path.join(analysis_dir, 'tracing_ref_400')
tracing = ['square','shape', 'this circle']

#### Set Up

In [ ]:
really_run_this = 0
if really_run_this:
    dir_name = 'tracing_overlap2'

    for sub_dir in os.listdir(dir_name):
        if sub_dir != '.DS_Store':
    #         for s in ['square', 'shape']:
    #         for s in ['circle']:
                s_dir = os.path.join(analysis_dir, dir_name, sub_dir, s)
                if os.path.exists(s_dir):
                    for f in os.listdir(s_dir):
                        if f.endswith('png'):
                            new_fname = f.split('_')
                            new_fname.insert(4, 'tracing')
                            new_fname = os.path.join(analysis_dir, dir_name, sub_dir, s, '_'.join(new_fname))
                            old_fname = os.path.join(analysis_dir, dir_name, sub_dir, s, f)
                            os.rename(old_fname, new_fname)
    

In [ ]:
really_run_this = 0
if really_run_this:
    # Overlap the drawing on reference images
    iteration_name = 'cdm_run_v4'# name labels used in kiddraw database
    dname = 'museumstation_v4' # name labels will be used for new data storage 
    tname = 'this square'

    # input folder
    current_data_dir = os.path.join(analysis_dir,"exp_data", "tracing_{}".format(iteration_name),tname)

    #output folder
    overlap_dir = os.path.join(analysis_dir, 'tracing_overlap2')
    overlap_t = os.path.join(overlap_dir, dname, tname)
    if not os.path.exists(overlap_t):
        os.makedirs(overlap_t)

    for fname in os.listdir(current_data_dir):
        if fname.startswith('age'):
            img_draw = os.path.join(current_data_dir, fname)
            img_ref = os.path.join(ref_dir, 'square_ref.png')

            draw = cv2.imread(img_draw).astype(int16)
            ref = cv2.imread(img_ref).astype(int16)
            img = ref
            img[np.sum(ref, axis=2)<255*3] = [150, 150, 150]
            img[np.sum(draw, axis=2)<255*3] = [40, 40, 170]

            split_fname = fname.split('_')
            age = split_fname[0]
            timestamp = split_fname[3][2:]
            new_fname = "{}_{}_{}_{}.png".format(age, dname, timestamp, 'copy_square')
            cv2.imwrite(os.path.join(overlap_t, new_fname), img)

#### upload stims to s3

Execute python script at command line by running: `python upload_stims_to_s3.py --bucket_name kiddraw-tracing --path_to_imgs ./tracing_eval_stims`

#### build stimulus dictionary & upload metadata to mongo

In [ ]:
# read in data from csv files
this_sq = pd.read_csv('exp_data/MuseumStation_Tracing_Descriptives_v4_this_square.csv')
limit = 300
data = []

for a in range(2,11):
    current = this_sq.loc[(this_sq['age'] == 'age{}'.format(a) )][:limit]
    data.append(current)

overall = pd.concat(data)
for a in range(2,11):
    print a, overall.loc[(overall['age'] == 'age{}'.format(a))].shape

overall = overall.drop(['filename', "Unnamed: 0"], axis=1)
overall.reset_index(inplace = True)

In [ ]:
overall['exp_name'] = 0
overall['iteration_name'] = 0
overall['timestamp'] = 0
overall['img_url'] = 0
overall['number_rating_levels'] = 5
overall['lower_bound'] = 'poor'
overall['upper_bound'] = 'excellent' 
overall['has_ref'] = False

category = 'copy_square'
for index, row in overall.iterrows():
    sid = row['session_id']    
    overall.loc[index,'exp_name'] = 'museumstation'
    overall.loc[index,'iteration_name'] = sid.split('_')[2][:2]
    
    timestamp = row['session_id'].split('_')[-1][2:]
    overall.loc[index, 'timestamp'] = timestamp
    
    fname = '{}_{}_{}_{}_{}.png'.format(row['age'], overall.loc[index,'exp_name'],overall.loc[index,'iteration_name'], timestamp, category)
    
    folder_name = '{}_{}'.format(overall.loc[index,'exp_name'], overall.loc[index,'iteration_name'])
    render_fname = os.path.join(data_dir, folder_name, category, fname)
    
    if not os.path.exists(render_fname):
        print index, render_fname
        overall = overall.drop(index)
        continue
    
    overall.loc[index,'img_url'] = 'https://s3.amazonaws.com/{}/{}'.format('kiddraw-tracing-2',fname)
    overall.loc[index,'games'] = '[]'
    overall.loc[index,'category'] = 'square'
    overall.loc[index,'age'] = overall.loc[index,'age'].replace('age','')

overall['age'] = pd.to_numeric(overall['age'])


In [ ]:
try:
    overall = overall.drop('index', axis=1)
except:
    pass

bin_limit = 80
for a in range(2,11):
    drop_index = overall.index[(overall['age'] == a)].tolist()[bin_limit:]
    overall = overall.drop(drop_index)
    print a, overall.loc[(overall['age'] == a)].shape



In [ ]:
## apply weird literal eval hack to convert string into regular list
import ast
overall['games'] = '[]'
overall['games'] = overall['games'].apply(lambda x: ast.literal_eval(x))
print overall.shape

In [ ]:
# insert shuffler_ind
data_range = np.arange(overall.shape[0])
np.random.shuffle(data_range)
overall['shuffler_ind'] = data_range

In [ ]:
try:
    overall = overall.drop('index', axis=1)
except:
    pass

## save out data frame as csv for our own records
overall.to_csv('kiddraw_tracing_eval_square_copy.csv',index=False)

In [ ]:
## convert 'overall' pandas dataframe to json object and save out
import json
stimdict = overall.to_dict(orient='records') 
with open('{}.js'.format('kiddraw_tracing_eval_square_copy'), 'w') as fout:
    json.dump(stimdict, fout)

### load JSON back in to inspect before uploading to mongo

In [2]:
### define datasetname
dataset_name = 'kiddraw_tracing_eval_square_copy' # production mode

### next todo is to upload this JSON to initialize the new stimulus collection
print('next todo is to upload this JSON to initialize the new stimulus collection...')
import json
J = json.loads(open('{}.js'.format(dataset_name),mode='ru').read())

next todo is to upload this JSON to initialize the new stimulus collection...


In [ ]:
##assert len(J)==len(all_files)
print 'dataset_name: {}'.format(dataset_name)
print len(J)

In [ ]:
## look at one example record
J[0]

In [3]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']

dev_name = 'kiddraw_tracing_eval_square_copy_dev' # development mode
coll = db[dataset_name]
dev_coll = db[dev_name]

In [ ]:
reallyRun = 0
if reallyRun:
    coll.drop()
print 'Currently there are {} tracings in the production collection'.format(coll.count())

In [ ]:
from IPython.display import clear_output
## actually add data now to the database
reallyRun = 1
if reallyRun:
    for (i,j) in enumerate(J):
        if i%1==0:
            print ('%d of %d' % (i,len(J)))
            clear_output(wait=True)
        coll.insert(j)

In [ ]:
reallyRun = 1
if reallyRun:
    dev_coll.drop()
print 'Currently there are {} tracings in the development collection'.format(dev_coll.count())

In [ ]:
# set the dev collection
reallyRun = 1
if reallyRun:
    for (i,j) in enumerate(J):
        if i%1==0:
            print ('%d of %d' % (i,len(J)))
            clear_output(wait=True)
        dev_coll.insert(j)

In [4]:
## check how many records have been retrieved
print 'Currently there are {} tracings in the production collection'.format(coll.count())
print 'Currently there are {} tracings in the development collection'.format(dev_coll.count())

a = coll.find({'num_strokes':{'$gte':0}}).sort('timestamp')
numGames = []
for rec in a:
    numGames.append(len(rec['games']))
b = np.array(numGames)
print np.mean(b>0)

Currently there are 713 tracings in the production collection
Currently there are 713 tracings in the development collection
0.0


In [5]:
print "Currentely, there are {} number of tracing stimuli in the database".format(coll.count())
coll.find_one()

Currentely, there are 713 number of tracing stimuli in the database


{u'_id': ObjectId('5c5c1b451943e60fb0c105bf'),
 u'age': 2,
 u'bounding_box': u'(0, 370, 83, 399)',
 u'category': u'square',
 u'draw_duration_new': 3.67,
 u'draw_duration_old': 0.0,
 u'exp_name': u'museumstation',
 u'games': [],
 u'has_ref': False,
 u'img_url': u'https://s3.amazonaws.com/kiddraw-tracing-2/age2_museumstation_v4_1536256584193_copy_square.png',
 u'iteration_name': u'v4',
 u'lower_bound': u'poor',
 u'mean_intensity': 0.09276875,
 u'num_strokes': 1,
 u'number_rating_levels': 5,
 u'session_id': u'cdm_run_v41536256584193',
 u'shuffler_ind': 234,
 u'submit_date': u'2018-09-06T18:08:31.999Z',
 u'submit_time': 1536257311999.0,
 u'timestamp': u'1536256584193',
 u'trial_duration': 7.61,
 u'trial_num': 3,
 u'upper_bound': u'excellent'}

In [6]:
dev_coll.find_one()

{u'_id': ObjectId('5c5c1b451943e60fb0c105bf'),
 u'age': 2,
 u'bounding_box': u'(0, 370, 83, 399)',
 u'category': u'square',
 u'draw_duration_new': 3.67,
 u'draw_duration_old': 0.0,
 u'exp_name': u'museumstation',
 u'games': [],
 u'has_ref': False,
 u'img_url': u'https://s3.amazonaws.com/kiddraw-tracing-2/age2_museumstation_v4_1536256584193_copy_square.png',
 u'iteration_name': u'v4',
 u'lower_bound': u'poor',
 u'mean_intensity': 0.09276875,
 u'num_strokes': 1,
 u'number_rating_levels': 5,
 u'session_id': u'cdm_run_v41536256584193',
 u'shuffler_ind': 234,
 u'submit_date': u'2018-09-06T18:08:31.999Z',
 u'submit_time': 1536257311999.0,
 u'timestamp': u'1536256584193',
 u'trial_duration': 7.61,
 u'trial_num': 3,
 u'upper_bound': u'excellent'}